In [1]:
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import seaborn as sns

%matplotlib inline 

In [2]:
#read in smileon data just in case it is needed:
smileon_full = pd.read_csv('../data2/SmileOn11.28.2020.csv', skiprows=2, encoding='ISO-8859-1')

C:\Users\kkosf\anaconda3\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32,36,53,59,88,89,127,141,144,156,157,158,159) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
health_ranks = pd.read_excel('../data/health_factors_ranks.xlsx', skiprows=1)
health_ranks.head()

,County,Rank
0,Anderson,17
1,Bedford,72
2,Benton,80
3,Bledsoe,93
4,Blount,9


In [4]:
health_ranks.sort_values('Rank')

,County,Rank
93,Williamson,1
94,Wilson,2
46,Knox,3
82,Sumner,4
74,Rutherford,5
...,...,...
37,Haywood,91
33,Hancock,92
3,Bledsoe,93
48,Lauderdale,94


In [5]:
inactivity = pd.read_csv('../data/adult_inactivity_2019.csv')
print(inactivity.head())
inactivity['County Value'] = inactivity['County Value'].str.rstrip('%').astype('int') / 100.0
inactivity.head()

     County County Value
0  Anderson          28%
1   Bedford          33%
2    Benton          32%
3   Bledsoe          29%
4    Blount          26%


,County,County Value
0,Anderson,0.28
1,Bedford,0.33
2,Benton,0.32
3,Bledsoe,0.29
4,Blount,0.26


In [6]:
pop = pd.read_csv('../data/tn_pop_county.csv')
print(pop.head())
MapLocSplit = pop["NAME"].str.rsplit(" ", n = 1, expand = True) 
pop["name"]= MapLocSplit[0] 
pop["county"]= MapLocSplit[1] 
pop["population"] = pop["population"].str.replace(",","").astype(int)
pop.head()

              NAME population
0  Anderson County     76,978
1   Bedford County     49,713
2    Benton County     16,160
3   Bledsoe County     15,064
4    Blount County    133,088


,NAME,population,name,county
0,Anderson County,76978,Anderson,County
1,Bedford County,49713,Bedford,County
2,Benton County,16160,Benton,County
3,Bledsoe County,15064,Bledsoe,County
4,Blount County,133088,Blount,County


In [7]:
pop_inactive = pd.merge(left = pop, right = inactivity, 
                    left_on = 'name', right_on = 'County')
print(pop_inactive.shape)
pop_inactive.head(3)

(95, 6)


,NAME,population,name,county,County,County Value
0,Anderson County,76978,Anderson,County,Anderson,0.28
1,Bedford County,49713,Bedford,County,Bedford,0.33
2,Benton County,16160,Benton,County,Benton,0.32


In [8]:
pop_inactive = pop_inactive[['County', 'population', 'County Value']]
pop_inactive

,County,population,County Value
0,Anderson,76978,0.28
1,Bedford,49713,0.33
2,Benton,16160,0.32
3,Bledsoe,15064,0.29
4,Blount,133088,0.26
...,...,...,...
90,Wayne,16673,0.38
91,Weakley,33328,0.32
92,White,27345,0.29
93,Williamson,238412,0.20


In [18]:
pop_inactive['total'] = (pop_inactive['population']*pop_inactive['County Value']).round(0).astype(int)
pop_inactive.to_csv(r'C:\Users\kkosf\Documents\nss\projects\smile-on-spiderman\pop_inactive.csv', index=False)
pop_inactive

,County,population,County Value,total
0,Anderson,76978,0.28,21554
1,Bedford,49713,0.33,16405
2,Benton,16160,0.32,5171
3,Bledsoe,15064,0.29,4369
4,Blount,133088,0.26,34603
...,...,...,...,...
90,Wayne,16673,0.38,6336
91,Weakley,33328,0.32,10665
92,White,27345,0.29,7930
93,Williamson,238412,0.20,47682


In [17]:
pop_inactive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 94
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   County        95 non-null     object 
 1   population    95 non-null     int32  
 2   County Value  95 non-null     float64
 3   total         95 non-null     int32  
dtypes: float64(1), int32(2), object(1)
memory usage: 3.0+ KB
